In [1]:
# import libraries
import pandas as pd
from selenium import webdriver
from bs4 import BeautifulSoup as bs
import requests
from lxml import html as lh
import traceback
from selenium.webdriver.common.keys import Keys
from selenium.webdriver.support import expected_conditions as EC
from selenium.webdriver.common.by import By
from selenium.webdriver.support.ui import WebDriverWait

### Settings for browser

In [2]:
# create options object for chromium webdriver
options = webdriver.ChromeOptions()
# setup path for brave binary (executable file)
options.binary_location = "C:\\Program Files (x86)\\BraveSoftware\\Brave-Browser\\Application\\brave.exe"
# add extensions for brave to open new instance of browser with VPN extension activated
# for this purpose I used extensions source code extractor on http://crxextractor.com/
# using link of extension found in store
options.add_extension(r"extension_4_1_0_0.crx")
# start chrome maximized
options.add_argument("start-maximized")
# downloaded chromium driver binaries and place it in project directory
chrome_driver_binary = "chromedriver.exe"

### Chrome Driver

In [3]:
# Chrome - controls browser
driver = webdriver.Chrome(chrome_driver_binary, options=options)
# choose which site you want to visit
driver.get("https://www.otodom.pl/sprzedaz/mieszkanie/wroclaw/?search%5Border%5D=quality_score&search%5Bcity_id%5D=39&nrAdsPerPage=72")
# check if it open right site by site title
assert "Mieszkanie na sprzedaż w Wrocław, dolnośląskie - www.otodom.pl" in driver.title

#### Get page content and parse it with beautifulSoup

In [152]:
content = driver.page_source
soup = bs(content)

##### Raw data is in html format, need to get raw text from tag then strip text and remove "zł" from data

In [32]:
offer_price = []
offer_id = set()
offer_room_number = []
offer_flat_sqm = []
offer_price_for_sqm = []
offer_floor = []
offer_floor_number = []
offer_prod_year = []
offer_rent = []
offer_market = []
offer_building_type = []
offer_finish_condition = []
offer_district = []

current_page = 0
page_number = 3

while current_page <= page_number:
    content = driver.page_source
    soup = bs(content)
    for article in soup.find_all("article"):
        article2 = []
        if article['id'] not in article2:
            
            article_id = article['id']
            article2.append(article_id)
            price = article.find('li', attrs={'class':'offer-item-price'})
            room_number = article.find('li', attrs={'class':'offer-item-rooms hidden-xs'})
            flat_sqm = article.find('li', attrs={'class':'hidden-xs offer-item-area'})
            price_for_sqm = article.find('li', attrs={'class':'hidden-xs offer-item-price-per-m'})

            offer_id.add(article_id)
            offer_price.append(price.get_text())
            offer_room_number.append(room_number.get_text())
            offer_flat_sqm.append(flat_sqm.get_text())
            offer_price_for_sqm.append(price_for_sqm.get_text())
        else:
            print('Oferta powtórzona')

    WebDriverWait(driver, 30).until(EC.element_to_be_clickable((By.CLASS_NAME, 'pager-next'))).click()
    current_page += 1

In [ ]:
for i in range(10):
    WebDriverWait(driver, 30).until(EC.element_to_be_clickable((By.XPATH, "//form[@id='pagerForm']/ul/li[@class='pager-next']"))).click()
    

In [40]:
# offer_price.append(price.get_text().replace("zł",'').replace(" ","").strip())
# cena = int(offer_price[1])
# cena2 = int(offer_price[2])
# cena+cena2
# print(offer_id)
print(offer_price)
# print(offer_room_number)
# print(offer_flat_sqm)
# print(offer_price_for_sqm)
# len(offer_price)
# len(offer_id)
# counter

['\n                                                                                    1 321 500 zł                                                                                                                                ', '\n                                                                                    324 268 zł                                                                                                                                ', '\n                                                                                    237 531 zł                                                                                                                                ', '\n                                                                                    359 000 zł                                                                                                                                ', '\n                                                                                  

## Scrap table data from gp-sms.pl
### Based on article from https://towardsdatascience.com/web-scraping-html-tables-with-python-c9baba21059

In [ ]:
options = webdriver.ChromeOptions()
options.binary_location = "C:/Program Files (x86)/BraveSoftware/Brave-Browser/Application/brave.exe"
chrome_driver_binary = "chromedriver.exe"
driver = webdriver.Chrome(chrome_driver_binary, options=options)
server_name = []
gold_amount = []

# elem = driver.find_element_by_class_name("table-gold-available")
driver.get("http://gp-sms.pl/")
content = driver.page_source
soup = bs(content)
doc = lh.fromstring(content)
tr_elements = doc.xpath('//tr')
#Create empty list
col=[]

#For each row, store each first element (header) and an empty list
for t in tr_elements[0]:
    name=t.text_content()
    col.append((name,[]))

#Since out first row is the header, data is stored on the second row onwards
for j in range(1,len(tr_elements)):
    #T is our j'th row
    T=tr_elements[j]
    
    #i is the index of our column
    i=0
    
    #Iterate through each element of the row
    for t in T.iterchildren():
        data=t.text_content() 
        #Check if row is empty
        if i > 0:
        #Convert any numerical value to integers
            try:
                data=int(data)
            except:
                pass
        #Append the data to the empty list of the i'th column
        col[i][1].append(data)
        #Increment i for the next column
        i+=1

In [ ]:
Dict={title:column for (title,column) in col}
df=pd.DataFrame(Dict)

In [ ]:
df.to_excel('gold_amount_gp_sms_scrap2.xlsx', index=False, encoding='utf-8')

In [ ]:
df.to_csv('gold_amount_gp_sms_scrap.csv', index=False, encoding='utf-8')

In [ ]:
df = pd.read_excel("gold_amount_gp_sms_scrap.xlsx", dtype={'Ilość CC': float})
df.to_excel('gold_amount_gp_sms_scrap2.xlsx', index=False, encoding='utf-8')

In [ ]:
df.head(100)

## Tutorial from selenium docs on custom example

In [ ]:
driver.get("http://gp-sms.pl/")
assert "Tibia GP za SMS" in driver.title
elem = driver.find_element_by_id("searchbar-gold-available")
elem.clear()
elem.send_keys("Harmonia")
elem.send_keys(Keys.RETURN)
assert "No results found." not in driver.page_source

## Tutorial from https://selenium-python.readthedocs.io/

In [ ]:
options = webdriver.ChromeOptions()
options.binary_location = "C:/Program Files (x86)/BraveSoftware/Brave-Browser/Application/brave.exe"
chrome_driver_binary = "chromedriver.exe"
driver = webdriver.Chrome(chrome_driver_binary, options=options)
driver.get("http://www.python.org")
assert "Python" in driver.title
elem = driver.find_element_by_name("q")
elem.clear()
elem.send_keys("dsafdaga")
elem.send_keys(Keys.RETURN)
assert "No results found." not in driver.page_source
# driver.close()